In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/heart (2).csv')

In [ ]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
df.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [ ]:
df.shape

(918, 12)

In [ ]:
df.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.dtypes

,0
Age,int64
Sex,object
ChestPainType,object
RestingBP,int64
Cholesterol,int64
FastingBS,int64
RestingECG,object
MaxHR,int64
ExerciseAngina,object
Oldpeak,float64


In [ ]:
numeric_cols = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak', 'HeartDisease']

outlier_counts = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_counts[col] = len(outliers)

for col, count in outlier_counts.items():
    print(f"{col}: {count} outliers")

Age: 0 outliers
RestingBP: 28 outliers
Cholesterol: 183 outliers
FastingBS: 214 outliers
MaxHR: 2 outliers
Oldpeak: 16 outliers
HeartDisease: 0 outliers


In [ ]:
from scipy.stats import zscore
z_scores = df[numeric_cols].apply(zscore)
df_cleaned = df[(abs(z_scores) < 3).all(axis=1)]

In [ ]:
for col in numeric_cols:
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_cleaned[(df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)]
    outlier_counts[col] = len(outliers)
for col, count in outlier_counts.items():
    print(f"{col}: {count} outliers")

Age: 0 outliers
RestingBP: 19 outliers
Cholesterol: 177 outliers
FastingBS: 209 outliers
MaxHR: 1 outliers
Oldpeak: 8 outliers
HeartDisease: 0 outliers


In [ ]:
df['HeartDisease'].unique()

array([0, 1])

In [ ]:
categorical_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
df_encoded = pd.get_dummies(df_cleaned, columns=categorical_cols, drop_first=True)

In [ ]:
X = df_encoded.drop('HeartDisease', axis=1)
y = df_encoded['HeartDisease']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
df['HeartDisease'].unique()

array([0, 1])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
df_encoded.shape

(899, 16)

In [ ]:
X_train.shape

(719, 15)

In [ ]:
X_test.shape

(180, 15)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Dense(128, input_dim=15, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', verbose=1, save_best_only=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stop, checkpoint])

Epoch 1/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5000 - loss: 0.6918
Epoch 1: val_loss improved from inf to 0.62039, saving model to best_model.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.5481 - loss: 0.6706 - val_accuracy: 0.7083 - val_loss: 0.6204
Epoch 2/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - accuracy: 0.6562 - loss: 0.6515
Epoch 2: val_loss improved from 0.62039 to 0.53899, saving model to best_model.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7246 - loss: 0.6119 - val_accuracy: 0.8194 - val_loss: 0.5390
Epoch 3/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8750 - loss: 0.4846
Epoch 3: val_loss improved from 0.53899 to 0.44726, saving model to best_model.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8473 - loss: 0.5045 - val_accuracy: 0.8403 - val_loss: 0.4473
Epoch 4/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.8281 - loss: 0.4379
Epoch 4: val_loss improved from 0.44726 to 0.37335, saving model to best_m

In [ ]:
loss, accuracy= model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8545 - loss: 0.3636 
Test Loss: 0.3512
Test Accuracy: 0.8778


In [ ]:
model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


array([[0.97378623],
       [0.94124806],
       [0.5967167 ],
       [0.93948764],
       [0.85141194],
       [0.6104491 ],
       [0.38444188],
       [0.93242085],
       [0.04541142],
       [0.9494057 ],
       [0.96276504],
       [0.01837224],
       [0.75813   ],
       [0.98300344],
       [0.850328  ],
       [0.1264711 ],
       [0.92357105],
       [0.13064426],
       [0.7955214 ],
       [0.8605954 ],
       [0.06680631],
       [0.9745082 ],
       [0.9240696 ],
       [0.08557778],
       [0.3064282 ],
       [0.93742406],
       [0.42467013],
       [0.02042372],
       [0.945948  ],
       [0.7841091 ],
       [0.8033095 ],
       [0.19467343],
       [0.03954317],
       [0.90051347],
       [0.92810446],
       [0.88850486],
       [0.933736  ],
       [0.03519523],
       [0.03372971],
       [0.72746533],
       [0.23689525],
       [0.99543905],
       [0.8675068 ],
       [0.61357   ],
       [0.6982597 ],
       [0.76105744],
       [0.01802824],
       [0.859

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

model2 = Sequential()
model2.add(Dense(16, input_dim=15, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='best_model2.keras', monitor='val_loss', verbose=1, save_best_only=True)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['recall'])

In [ ]:
history = model2.fit(X_train, y_train, epochs=60, batch_size=64, validation_split=0.2, callbacks=[early_stop, checkpoint])

Epoch 1/60
1/9 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.6887 - recall: 0.9429
Epoch 1: val_loss improved from inf to 0.66832, saving model to best_model2.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.6831 - recall: 0.9726 - val_loss: 0.6683 - val_recall: 1.0000
Epoch 2/60
1/9 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step - loss: 0.6641 - recall: 1.0000
Epoch 2: val_loss improved from 0.66832 to 0.64568, saving model to best_model2.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6605 - recall: 0.9949 - val_loss: 0.6457 - val_recall: 0.9872
Epoch 3/60
1/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.6270 - recall: 1.0000
Epoch 3: val_loss improved from 0.64568 to 0.61593, saving model to best_model2.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6297 - recall: 0.9811 - val_loss: 0.6159 - val_recall: 0.9744
Epoch 4/60
1/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.6042 - recall: 1.0000
Epoch 4: val_loss improved from 0.61593 to 0.57617, saving model to best_model2.keras
9/9 ━━

In [ ]:
loss, accuracy= model2.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3725 - recall: 0.9242 
Test Loss: 0.3616
Test Accuracy: 0.9362


In [ ]:
model2.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


array([[0.8892877 ],
       [0.94167626],
       [0.38749456],
       [0.95125455],
       [0.87808955],
       [0.6031339 ],
       [0.35518044],
       [0.8726725 ],
       [0.04982502],
       [0.96459466],
       [0.9656652 ],
       [0.03501344],
       [0.7875262 ],
       [0.9768503 ],
       [0.83623004],
       [0.24037594],
       [0.88770294],
       [0.18928638],
       [0.73968595],
       [0.9208291 ],
       [0.04974295],
       [0.97602093],
       [0.85428655],
       [0.26391423],
       [0.26825553],
       [0.9437097 ],
       [0.3478307 ],
       [0.03217115],
       [0.94792336],
       [0.62816685],
       [0.8268721 ],
       [0.16956393],
       [0.05242353],
       [0.8798347 ],
       [0.2079404 ],
       [0.7804562 ],
       [0.85100126],
       [0.08010194],
       [0.03213158],
       [0.7884892 ],
       [0.21957465],
       [0.97104734],
       [0.8590952 ],
       [0.4648107 ],
       [0.9482029 ],
       [0.4001722 ],
       [0.0256737 ],
       [0.865

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

model3 = Sequential()
model3.add(Dense(16, input_dim=15, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='best_model3.keras', monitor='val_loss', verbose=1, save_best_only=True)
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['precision'])

In [ ]:
history = model3.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stop, checkpoint])

Epoch 1/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - loss: 0.6983 - precision: 0.4118
Epoch 1: val_loss improved from inf to 0.67117, saving model to best_model3.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.6905 - precision: 0.5329 - val_loss: 0.6712 - val_precision: 0.5662
Epoch 2/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - loss: 0.6585 - precision: 0.6500
Epoch 2: val_loss improved from 0.67117 to 0.63657, saving model to best_model3.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6583 - precision: 0.6072 - val_loss: 0.6366 - val_precision: 0.6271
Epoch 3/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.6174 - precision: 0.7059
Epoch 3: val_loss improved from 0.63657 to 0.57442, saving model to best_model3.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6175 - precision: 0.6768 - val_loss: 0.5744 - val_precision: 0.7500
Epoch 4/50
1/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.5929 - precision: 0.7317
Epoch 4: val_loss improved from 0.57442 to 0.49223, saving mod

In [ ]:
loss, accuracy= model2.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3566 - recall: 0.9556 
Test Loss: 0.3376
Test Accuracy: 0.9574
